In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from datetime import timedelta

In [2]:
def f_unique_len_sum(df_payments, metrix):
# def f_unique_len_sum(df_payments, test_groups, metrix):
    test_groups = ['TG','CG1','CG2']
    out = []
    for test_group in test_groups:
        out.append(len(set(df_payments[(df_payments['test group'] == test_group) & (df_payments[metrix] == True)]['user ID'])))
    for test_group in test_groups:
        out.append(len(df_payments[(df_payments['test group'] == test_group) & (df_payments[metrix] == True)]['user ID']))
    for test_group in test_groups:
        out.append(df_payments[(df_payments['test group'] == test_group) & (df_payments[metrix] == True)]['revenue in cents'].sum())
    idx = ['payers__TG','payers__CG1','payers__CG2','payments__TG','payments__CG1','payments__CG2','revenue__TG','revenue__CG1','revenue__CG2']
    return(pd.DataFrame(out, columns = [metrix], index = idx))

In [3]:
# https://portraits.core.tf/app/kibana#/visualize/create?type=histogram&indexPattern=portraits&_g=(refreshInterval:(display:Off,pause:!f,value:0),time:(from:now-15m,mode:quick,to:now))&_a=(filters:!(),linked:!f,query:(query_string:(analyze_wildcard:!t,query:'*')),uiState:(),vis:(aggs:!((id:'1',params:(),schema:metric,type:count),(id:'2',params:(filters:!(('$$hashKey':'object:3287',input:(query:(query_string:(analyze_wildcard:!t,query:'tags:Multirun_FullPremListWithoutWeek_Android_run1__TG'))),label:''),('$$hashKey':'object:3296',input:(query:(query_string:(analyze_wildcard:!t,query:'tags:Multirun_FullPremListWithoutWeek_Android_run1__CG1')))),('$$hashKey':'object:3301',input:(query:(query_string:(analyze_wildcard:!t,query:'tags:Multirun_FullPremListWithoutWeek_Android_run1__CG2')))))),schema:segment,type:filters),(id:'3',params:(field:country,order:desc,orderBy:'1',size:200),schema:group,type:terms),(id:'4',params:(filters:!(('$$hashKey':'object:3189',input:(query:(query_string:(analyze_wildcard:!t,query:'referral:0'))),label:''),('$$hashKey':'object:3198',input:(query:(query_string:(analyze_wildcard:!t,query:'referral:1'))))),row:!t),schema:split,type:filters)),listeners:(),params:(addLegend:!t,addTimeMarker:!f,addTooltip:!t,defaultYExtents:!f,mode:stacked,scale:linear,setYExtents:!f,shareYAxis:!t,times:!(),yAxis:()),title:'New%20Visualization',type:histogram))
df_installs = pd.read_csv('Multirun_FullPremListWithoutWeek_Android_run1_inst.csv')
df_payments_TG = pd.read_csv('Multirun_FullPremListWithoutWeek_Android_run1__TG.csv')
df_payments_CG1 = pd.read_csv('Multirun_FullPremListWithoutWeek_Android_run1__CG1.csv')
df_payments_CG2 = pd.read_csv('Multirun_FullPremListWithoutWeek_Android_run1__CG2.csv')

In [4]:
df_payments_TG['test group'] = 'TG'
df_payments_CG1['test group'] = 'CG1'
df_payments_CG2['test group'] = 'CG2'
df_payments = pd.concat([df_payments_TG,df_payments_CG1,df_payments_CG2])

In [5]:
df_payments['Product Id'] = df_payments['Product Id'].fillna('null')
df_payments['is_prem'] = df_payments['Product Id'].map(lambda x: bool(bool(re.search('prem',x)) + bool(re.search('vip',x)) + bool(re.search('trial',x))))
df_payments['is_trial'] = df_payments['Product Id'].map(lambda x:  bool(re.search('trial',x)) )
df_payments['is_coin'] = df_payments['Product Id'].map(lambda x:  bool(re.search('coin',x)) )
df_payments['is_like'] = df_payments['Product Id'].map(lambda x:  bool(re.search('like',x)) )
df_payments['is_bomb'] = df_payments['Product Id'].map(lambda x:  bool(re.search('bomb',x)) )

In [6]:
df_payments.head()

,ref ID,partner ID,ref date,user ID,registration date,payment ID,payment date,payment amount,revenue in cents,payment service ID,...,place,country,last app,Product Id,test group,is_prem,is_trial,is_coin,is_like,is_bomb
0,9485.0,63.0,2018-04-02 17:24:06,117622766,2018-04-02 17:24:05,1,2018-04-02 17:31:51,516,309,19,...,none,DE,topfacemobappandroid,com.topface.topface.buy.sms.coins.30,TG,False,False,True,False,False
1,8786.0,42.0,2018-04-01 13:23:29,117587116,2018-04-01 13:21:17,24,2018-04-01 19:47:27,100,97,19,...,autorefill,UA,topfacemobappandroid,pn.coins.6.1,TG,False,False,True,False,False
2,NaN,NaN,NaN,117710948,2018-04-06 15:21:56,3,2018-04-09 18:12:30,984,688,147,...,TrialVipPopup,TR,topfacemobappandroid,com.topface.topface.sub.trial.vip.month.13.3da...,TG,True,True,False,False,False
3,9602.0,63.0,2018-04-06 06:09:58,117705619,2018-04-06 06:09:57,3,2018-04-10 23:27:59,300,291,67,...,Gifts,CM,topfacemobappandroid,pn.likes.100.3,TG,False,False,False,True,False
4,NaN,NaN,NaN,117702090,2018-04-05 23:53:22,1,2018-04-06 21:15:04,561,392,147,...,LeftMenu,TR,topfacemobappandroid,com.topface.topface.sub.vip.month.10,TG,True,False,False,False,False


In [7]:
# df_payments = df_payments[df_payments['ref ID'].isna()]
# df_payments = df_payments[~df_payments['ref ID'].isna()]

# df_payments = df_payments[df_payments['registration date'] < '2018-03-24']

In [8]:
df_payments['all'] = True
metrices = ['all','is_prem','is_trial','is_coin','is_like','is_bomb']

In [9]:
spend_by_types = pd.concat(list(map(lambda x: f_unique_len_sum(df_payments, x), metrices)), axis = 1)

In [10]:
spend_by_types

,all,is_prem,is_trial,is_coin,is_like,is_bomb
payers__TG,263,230,124,66,38,1
payers__CG1,331,291,133,85,47,2
payers__CG2,298,262,119,75,44,0
payments__TG,418,232,124,142,43,1
payments__CG1,496,293,133,140,57,6
payments__CG2,453,271,119,121,61,0
revenue__TG,282535,234908,121198,38497,9059,71
revenue__CG1,273836,215900,118791,46247,10802,887
revenue__CG2,242756,191870,106843,38571,12315,0


In [16]:
pd.concat(list(map(lambda x: f_unique_len_sum(df_payments[~(df_payments['ref ID'] > 0)], x), metrices)), axis = 1)

,all,is_prem,is_trial,is_coin,is_like,is_bomb
payers__TG,54,52,25,11,5,0
payers__CG1,46,42,21,11,6,1
payers__CG2,39,38,20,3,4,0
payments__TG,72,52,25,15,5,0
payments__CG1,72,43,21,17,7,5
payments__CG2,51,39,20,6,6,0
revenue__TG,45093,40990,24485,3197,906,0
revenue__CG1,47913,41361,20243,5065,879,608
revenue__CG2,41755,39210,18269,910,1635,0


In [17]:
pd.concat(list(map(lambda x: f_unique_len_sum(df_payments[df_payments['ref ID'] > 0], x), metrices)), axis = 1)

,all,is_prem,is_trial,is_coin,is_like,is_bomb
payers__TG,209,178,99,55,33,1
payers__CG1,285,249,112,74,41,1
payers__CG2,259,224,99,72,40,0
payments__TG,346,180,99,127,38,1
payments__CG1,424,250,112,123,50,1
payments__CG2,402,232,99,115,55,0
revenue__TG,237442,193918,96713,35300,8153,71
revenue__CG1,225923,174539,98548,41182,9923,279
revenue__CG2,201001,152660,88574,37661,10680,0
